In [71]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np

In [35]:
artists_page = 'https://en.wikipedia.org/wiki/2_Chainz_discography'

In [36]:
page = requests.get(artists_page)

In [37]:
soup = BeautifulSoup(page.text,'html.parser')

In [38]:
th = soup.find_all("th")

In [39]:
aa= []
for ii in th:
    aa.append(soup.find_all('a'))

In [40]:
scope = soup.find_all(re.compile(r'th'))

In [41]:
artist_releases = []
for i in scope:
    q = i.find_all('i')
    artist_releases.append(q)
    

In [42]:
artist_releases

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [<i><a href="/wiki/Based_on_a_T.R.U._Story" title="Based on a T.R.U. Story">Based on a T.R.U. Story</a></i>],
 [<i><a href="/wiki/B.O.A.T.S._II:_Me_Time" title="B.O.A.T.S. II: Me Time">B.O.A.T.S. II: Me Time</a></i>],
 [<i><a href="/wiki/ColleGrove" title="ColleGrove">ColleGrove</a></i>],
 [<i><a href="/wiki/Pretty_Girls_Like_Trap_Music" title="Pretty Girls Like Trap Music">Pretty Girls Like Trap Music</a></i>],
 [<i><a href="/wiki/Rap_or_Go_to_the_League" title="Rap or Go to the League">Rap or Go to the League</a></i>],
 [],
 [],
 [],
 [],
 [<i>Me Against the World</i>],
 [<i>All Ice on Me</i>],
 [<i>Trap-A-Velli</i>],
 [<i>Me Against the World 2: Codeine<br/>Withdrawal</i>],
 [<i>Trap-A-Velli 2: (The Residue)</i>],
 [<i>Codeine Cowboy (A 2 Chainz<br/>Collective)</i>],
 [<i><a href="/wiki/T.R.U._REALigion" title="T.R.U. REALigion">T.R.U. REALigion</a></i>],
 [<i>T.R.U. Jack City</i>],


In [10]:
artist_releases = [ item for sublist in artist_releases for item in sublist ]

In [57]:
def get_artist_releases(url):
    page = requests.get('https://en.wikipedia.org'+url+"_discography")
    soup = BeautifulSoup(page.text,'html.parser')
    scope = soup.find_all(re.compile(r'th'))
    artist_releases = []
    for i in scope:
        q = i.find_all('i')
        artist_releases.append(q)
    artist_releases = [ item for sublist in artist_releases for item in sublist ]
    triplet_list= []
    for i in artist_releases:
        try:
            triplet_list.append( [ i.a['href'], i.a['title'], url])
        except:
            pass
    return triplet_list

In [58]:
artist_releases = get_artist_releases('/wiki/2_Chainz')
artist_releases

[['/wiki/Based_on_a_T.R.U._Story',
  'Based on a T.R.U. Story',
  '/wiki/2_Chainz'],
 ['/wiki/B.O.A.T.S._II:_Me_Time', 'B.O.A.T.S. II: Me Time', '/wiki/2_Chainz'],
 ['/wiki/ColleGrove', 'ColleGrove', '/wiki/2_Chainz'],
 ['/wiki/Pretty_Girls_Like_Trap_Music',
  'Pretty Girls Like Trap Music',
  '/wiki/2_Chainz'],
 ['/wiki/Rap_or_Go_to_the_League',
  'Rap or Go to the League',
  '/wiki/2_Chainz'],
 ['/wiki/T.R.U._REALigion', 'T.R.U. REALigion', '/wiki/2_Chainz'],
 ['/wiki/Daniel_Son;_Necklace_Don',
  'Daniel Son; Necklace Don',
  '/wiki/2_Chainz'],
 ['/wiki/Hibachi_for_Lunch', 'Hibachi for Lunch', '/wiki/2_Chainz'],
 ['/wiki/The_Play_Don%27t_Care_Who_Makes_It',
  "The Play Don't Care Who Makes It",
  '/wiki/2_Chainz'],
 ['/wiki/Wikipedia:Citation_needed',
  'Wikipedia:Citation needed',
  '/wiki/2_Chainz']]

In [64]:
artists = pd.read_csv("rappers_with_wikipedia_pages.csv",index_col=0)

In [65]:
wiki_artists= artists[artists["status_code"]==200]

In [59]:
all_releases=wiki_artists["wikipedia_page"].apply(lambda x: get_artist_releases(x) )

In [90]:
wiki_artists["wikipedia_page"][3]

'/wiki/2_Chainz'

In [67]:
rl = [ item for sublist in all_releases for item in sublist ]

In [72]:
rl = np.stack(rl)

In [75]:
rl

array([['/wiki/Based_on_a_T.R.U._Story', 'Based on a T.R.U. Story',
        '/wiki/2_Chainz'],
       ['/wiki/B.O.A.T.S._II:_Me_Time', 'B.O.A.T.S. II: Me Time',
        '/wiki/2_Chainz'],
       ['/wiki/ColleGrove', 'ColleGrove', '/wiki/2_Chainz'],
       ...,
       ['/wiki/Unknown_Death_2002', 'Unknown Death 2002',
        '/wiki/Yung_Lean'],
       ['/wiki/Frost_God', 'Frost God', '/wiki/Yung_Lean'],
       ['/wiki/Lavender_EP', 'Lavender EP', '/wiki/Yung_Lean']],
      dtype='<U121')

In [80]:
release_df = pd.DataFrame(columns=["release_wiki", "release_name", "wikipedia_page"],data = rl)

In [91]:
type(release_df["wikipedia_page"][0]), type(wiki_artists["wikipedia_page"][3])

(str, str)

In [95]:
release_df = release_df.merge(wiki_artists[["name","wikipedia_page"]], on=["wikipedia_page"])

In [96]:
release_df.to_csv("artist_and_albums.csv",index=False)